# This is the notebook for my Coursera IBM Datascience Capstone project

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


To fix: convert values in the selected features to be a float.

# Introduction/Business Problem

I am looking to determine the probability of a car accident occuring as well as the severity of the accident. I will be utilizing weather, traffic, and road contidion data in order to determine these two factors. My target audience will be car drivers who would like to know this information in order to be safer on the road.

# Data

I will be using the data from the csv provided for my predictions. I will definately be utilizing the road contidtions, weather, and severity. I will be utilizing AI trained on the sample data in order to predict the results. A simple at a glance example would be if it is dark outside, in weather that has bad visibility, at night (which also has bad visiblity), with wet/icy road contitions, then the results should show a high probability of an accident, with the severity being either "Property damage only" or "Injury Collision"

# PLAYING WITH RANDOM THINGS, WILL BE FIXED LATER

In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

print('Done!')

Done!


In [3]:
!wget -O collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-08-26 16:05:28--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘collisions.csv’

100%[======================================>] 73,917,638  46.0MB/s   in 1.5s   

2020-08-26 16:05:30 (46.0 MB/s) - ‘collisions.csv’ saved [73917638/73917638]



In [4]:
df = pd.read_csv('collisions.csv')
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [5]:
df.shape

(194673, 38)

### Data Visualization and Preprocessing

In [6]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [7]:
df['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [8]:
df['LIGHTCOND'].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [9]:
df.groupby(['LIGHTCOND'])['SEVERITYDESC'].value_counts()

LIGHTCOND                 SEVERITYDESC                  
Dark - No Street Lights   Property Damage Only Collision     1203
                          Injury Collision                    334
Dark - Street Lights Off  Property Damage Only Collision      883
                          Injury Collision                    316
Dark - Street Lights On   Property Damage Only Collision    34032
                          Injury Collision                  14475
Dark - Unknown Lighting   Property Damage Only Collision        7
                          Injury Collision                      4
Dawn                      Property Damage Only Collision     1678
                          Injury Collision                    824
Daylight                  Property Damage Only Collision    77593
                          Injury Collision                  38544
Dusk                      Property Damage Only Collision     3958
                          Injury Collision                   1944
Other              

In [10]:
#!conda install -c anaconda seaborn -y

In [11]:
#import seaborn as sns

In [12]:
stuff = df[['SEVERITYCODE', 'ROADCOND', 'LIGHTCOND' , 'WEATHER']]
stuff.head()

,SEVERITYCODE,ROADCOND,LIGHTCOND,WEATHER
0,2,Wet,Daylight,Overcast
1,1,Wet,Dark - Street Lights On,Raining
2,1,Dry,Daylight,Overcast
3,1,Dry,Daylight,Clear
4,2,Wet,Daylight,Raining


### Trying Decision tree because why not?

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
print('Decision tree installed!')

Decision tree installed!


In [14]:
feat = df[['ROADCOND', 'LIGHTCOND' , 'WEATHER']]
feat.head()

,ROADCOND,LIGHTCOND,WEATHER
0,Wet,Daylight,Overcast
1,Wet,Dark - Street Lights On,Raining
2,Dry,Daylight,Overcast
3,Dry,Daylight,Clear
4,Wet,Daylight,Raining


In [15]:
x = feat
x[0:5]

,ROADCOND,LIGHTCOND,WEATHER
0,Wet,Daylight,Overcast
1,Wet,Dark - Street Lights On,Raining
2,Dry,Daylight,Overcast
3,Dry,Daylight,Clear
4,Wet,Daylight,Raining


In [16]:
y = df['SEVERITYDESC'].values
y[0:5]

array(['Injury Collision', 'Property Damage Only Collision',
       'Property Damage Only Collision', 'Property Damage Only Collision',
       'Injury Collision'], dtype=object)

In [17]:
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=4)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (155738, 3) (155738,)
Test set: (38935, 3) (38935,)


In [18]:
'''x= preprocessing.StandardScaler().fit(x).transform(x)
x[0:5]'''

'x= preprocessing.StandardScaler().fit(x).transform(x)\nx[0:5]'

In [19]:
'''decTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
decTree.fit(x_train,y_train)

predict = decTree.predict(x_test)
print(predict[0:5])
print(y_test[0:5])'''

'decTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)\ndecTree.fit(x_train,y_train)\n\npredict = decTree.predict(x_test)\nprint(predict[0:5])\nprint(y_test[0:5])'

In [20]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

# All requested packages already installed.

Folium installed and imported!


In [44]:
crashloc = df[['X', 'Y']]
crashloc = crashloc.dropna()
crashloc.head()
crashloc = crashloc[:10]
#print(crashloc.X)

0   -122.323148
1   -122.347294
2   -122.334540
3   -122.334803
4   -122.306426
5   -122.387598
6   -122.338485
7   -122.320780
8   -122.335930
9   -122.384700
Name: X, dtype: float64


In [55]:
# define the world map
crashmap = folium.Map(location = [47.606, -122.332], zoom_start = 12)

# display world map
#crashmap #commenting out because i don't need a massive map showing up all the time

In [56]:
crash = folium.map.FeatureGroup()
for lat, lng, in zip(crashloc.Y, crashloc.X):
    crash.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

crashmap.add_child(crash)
print('done!')

done!


In [57]:
crashmap